# Tester KNN sur un nouveau jeu de données

La méthode vue dans le dernier chapitre peut être appliquée à n'importe quelle type de problème, pourvu qu'on possède des données desquelles on peut apprendre.


**Exercice** : on va essayer de deviner la *console de jeu* sur laquelle un jeu est sorti, à partir des informations suivantes :

- L'année de sortie du jeu (`Year`)
- Le nombre de copies du jeu vendues pour l'Amérique du Nord (`NA_Sales`), l'Europe (`EU_Sales`), le Japon (`JP_Sales`), et ailleurs (`Other_Sales`)

Ces données ont été extraites du site web https://www.vgchartz.com/ et mises disponibles ici: https://www.kaggle.com/datasets/arslanali4343/sales-of-video-games

J'ai fait une passe de modifications dessus pour vous éviter de faire le nettoyage de zéro (retrait de colonnes inutiles, gestion des données mal entrées ou manquantes, etc)

&nbsp;

Suivez les étapes dans les commentaires, et inspirez-vous du code montré plus haut.

In [27]:
#@title Importations
import numpy as np
import pandas as pd
import plotly.express as px

In [28]:
#@title Importer le jeu de données
df_jeux = pd.read_csv("https://raw.githubusercontent.com/316k/misc-data/master/vgsales-simple.csv", sep=',')

**QUESTION**

Avant de commencer, c'est bon de regarder les données qu'on a...

On va essayer de prédire la console de jeu à partir de l'année de sortie et
des chiffres de ventes.

Regardez le `DataFrame` et dites :
- Combien y a-t-il de jeux considérés au total?
- Combien y a-t-il de plateformes différentes?
- Quelle est l'année minimum et l'année maximum?
- Quelles sont les trois consoles de jeu qui reviennent le plus souvent dans le jeu de données?

In [29]:
#@title Exercice 1.1

# On regarde le jeu de données en général
display(df_jeux)
display(df_jeux.describe())


valeurs = df_jeux.Platform.value_counts()
display(valeurs)

print("Nombre de plateformes différentes:", valeurs.size)

,Name,Platform,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Wii Sports,Wii,2006,41490000.0,29020000.0,3770000.0,8460000.0
1,Super Mario Bros.,NintendoNES,1985,29080000.0,3580000.0,6810000.0,770000.0
2,Mario Kart Wii,Wii,2008,15850000.0,12880000.0,3790000.0,3310000.0
3,Wii Sports Resort,Wii,2009,15750000.0,11010000.0,3280000.0,2960000.0
4,Pokemon Red/Pokemon Blue,GameBoy,1996,11270000.0,8890000.0,10220000.0,1000000.0
...,...,...,...,...,...,...,...
2339,Grand Kingdom,PlayStationVita,2015,10000.0,20000.0,40000.0,10000.0
2340,Guilty Gear Xrd -Revelator-,PlayStation3,2016,10000.0,20000.0,20000.0,10000.0
2341,Odin Sphere: Leifthrasir,PlayStation3,2016,10000.0,20000.0,30000.0,10000.0
2342,Natural Doctrine,PlayStation3,2014,30000.0,10000.0,10000.0,10000.0


,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales
count,2344.000000,2.344000e+03,2.344000e+03,2.344000e+03,2.344000e+03
mean,2006.604096,9.067577e+05,5.773891e+05,3.076323e+05,1.842961e+05
std,6.530620,1.911187e+06,1.173397e+06,6.748532e+05,4.416202e+05
min,1983.000000,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
25%,2003.000000,1.400000e+05,7.000000e+04,3.000000e+04,3.000000e+04
50%,2008.000000,3.700000e+05,2.200000e+05,9.000000e+04,7.000000e+04
75%,2011.000000,9.400000e+05,6.100000e+05,2.800000e+05,1.800000e+05
max,2016.000000,4.149000e+07,2.902000e+07,1.022000e+07,1.057000e+07


,count
Platform,
PlayStation3,429
PlayStation2,288
XBox360,287
PlayStation,191
NintendoDS,160
Wii,147
PlayStation4,131
Nintendo3DS,114
PlayStationPortable,110


Nombre de plateformes différentes: 22


**RÉPONSES**

- 2344 Jeux (le nombre de lignes)
- 22 Plateformes
- Les années des jeux vont de 1983 à 2016 (colonne `Year`, min et max)
- Les trois consoles qui reviennent le plus souvent sont : PlayStation3, PlayStation2, XBox360

**QUESTION**

Affichez les graphiques suivants :

- Un histogramme pour la colonne de l'année de parution (`Year`)
- Un histogramme pour la colonne du nombre de ventes Nord-Américaines (`NA_Sales`)
- Un diagramme circulaire pour voir les proportions des différentes consoles (`Platform`) représentées dans le DataFrame.

In [30]:
#@title Exercice 1.2
display(px.histogram(df_jeux, x='Year'))
display(px.histogram(df_jeux, x='NA_Sales'))

valeurs = df_jeux.Platform.value_counts()
display(px.pie(valeurs, names=valeurs.index, values=valeurs.values))

## Entraînement d'un KNN sur ce jeu de données

In [48]:
#@title Exercice 2.1

# Mélangez les lignes du DataFrame au hasard. Utilisez random_state=60
df_jeux_melange = ...

######

df_jeux_melange = df_jeux.sample(frac=1, random_state=60)

### Séparez le DataFrame en deux :
### 1. df_entrainement, qui contient 2/3 des lignes
### 2. df_test, qui contient le dernier 1/3 des lignes
index_deux_tiers = int(len(df_jeux_melange) * 2/3)
df_entrainement = df_jeux_melange.iloc[:index_deux_tiers]
df_test = df_jeux_melange.iloc[index_deux_tiers:]

### Affichez df_entrainement et df_test pour voir le résultat
display(df_entrainement)
display(df_test)

,Name,Platform,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1021,Wario Land: Shake It!,Wii,2008,600000.0,310000.0,150000.0,110000.0
234,Halo 5: Guardians,XBoxOne,2015,2640000.0,1200000.0,30000.0,390000.0
1153,NES Open Tournament Golf,NintendoNES,1991,410000.0,100000.0,470000.0,20000.0
716,Xenosaga Episode I: Der Wille zur Macht,PlayStation2,2002,630000.0,490000.0,450000.0,170000.0
324,Resident Evil 5,XBox360,2009,2110000.0,940000.0,120000.0,340000.0
...,...,...,...,...,...,...,...
1714,Puzzle & Dragons Z + Super Mario Bros. Edition,Nintendo3DS,2015,50000.0,50000.0,340000.0,10000.0
489,PES 2009: Pro Evolution Soccer,PlayStation2,2008,130000.0,70000.0,260000.0,2050000.0
546,Sonic Advance,GameBoyAdvance,2001,1190000.0,710000.0,220000.0,130000.0
743,Metroid Fusion,GameBoyAdvance,2002,1180000.0,270000.0,170000.0,60000.0


,Name,Platform,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1816,LocoRoco,PlayStationPortable,2006,140000.0,50000.0,160000.0,40000.0
1793,No More Heroes 2: Desperate Struggle,Wii,2010,210000.0,120000.0,40000.0,30000.0
337,LittleBigPlanet 2,PlayStation3,2011,1820000.0,1070000.0,60000.0,470000.0
805,Dishonored,PlayStation3,2012,720000.0,540000.0,40000.0,250000.0
1707,Mario & Sonic at the Rio 2016 Olympic Games,Nintendo3DS,2016,40000.0,160000.0,240000.0,10000.0
...,...,...,...,...,...,...,...
2169,Warriors: Legends of Troy,PlayStation3,2011,60000.0,50000.0,70000.0,20000.0
72,Animal Crossing: New Leaf,Nintendo3DS,2012,2010000.0,2320000.0,4360000.0,410000.0
2147,The Witch and the Hundred Knight,PlayStation3,2013,60000.0,20000.0,100000.0,10000.0
1418,Metal Gear Solid: Portable Ops Plus,PlayStationPortable,2007,260000.0,10000.0,260000.0,170000.0


In [49]:
#@title Exercice 2.2

### Créez un modèle KNN nommé modele_jeux, avec une valeur de k = 5
modele_jeux = ...

########
# Solution
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

modele_jeux = KNeighborsClassifier(5)

### Séparez le DataFrame df_entrainement en :
### X_entrainement: toutes les caractéristiques, sauf le nom du jeu et sauf celle qu'on veut prédire
### y_entrainement: seulement la colonne qu'on veut prédire (la console de jeux)
X_entrainement = df_entrainement[['Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
y_entrainement = df_entrainement['Platform']

### Faites l'entrainement KNN du modèle avec fit()
modele_jeux.fit(X_entrainement, y_entrainement)

### Séparez le DataFrame df_test en deux de la même façon, X_test et y_test
X_test = df_test[['Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
y_test = df_test['Platform']

### Demandez au modèle de faire KNN ses prédictions
y_predictions = modele_jeux.predict(X_test)

### Calculez l'exactitude du modèle (l'accuracy) que vous avez obtenu
print("Exactitude:", accuracy_score(y_test, y_predictions))

Exactitude: 0.3350383631713555


Si vous avez tout fait correctement, vous devriez terminer avec une exactitude de **≈ 33.5%**

## Évaluer un modèle selon son contexte

Le **33.5%** de bonnes réponses qu'on a obtenues ici ne devrait pas être considéré dans l'absolu.

Ça serait une erreur de regarder 33.5% et de comparer ça à "Et si j'avais eu cette note dans un examen?"

Selon ce qu'on essaie de faire, 33.5% peut être bon.

&nbsp;

Si je vous disais que j'avais un algo qui prend en paramètre le *nom*
d'une personne et qui sait deviner la **couleur des yeux** avec **80%
d'exactitude**, est-ce que ça serait bon?

&nbsp;

Voici mon code :

```python
def couleur_des_yeux(nom):
  return "bruns"
```

Selon
[VisionDirect.fr](https://www.visiondirect.fr/blog/couleur-des-yeux)...

> *La couleur des yeux la plus courante est le marron, pour environ
> 80% de la population mondiale.*

Mon algorithme est exact 80% du temps...

&nbsp;

À l'inverse, si je vous disais que j'ai un algorithme capable de
prédire avec **95% d'exactitude** si une personne est rousse juste en
regardant son numéro de DA :

```python
def personne_est_rousse(numeroDA):
  ...
```

Est-ce que ça serait bon?

&nbsp;

Seulement 1 à 2% de la population est rousse... On pourrait avoir un score de 98% juste en disant toujours `faux`!

95%, ça serait très mauvais ici!

## Comparaison de notre modèle avec un algorithme simpliste (*baseline*)

33.5% de bonnes réponses, c'est un peu décevant, mais on devrait remettre ça en perspective : on va comparer ça à deux algorithmes naïfs pour nous donner une meilleure idée de ce que 33.5% représente.

&nbsp;

On va comparer notre modèle avec deux méthodes stupides :

1. Dire des choses au hasard
2. Toujours dire la réponse la plus commune

**QUESTION**

Si on avait répondu au hasard à chaque prédiction, quelle aurait été l'exactitude (*accuracy*) de nos réponses?

Répétez l'expérience 10 fois l'expérience suivante :
- Créez un tableau `y_predictions_hasard` qui contient un nom de console au hasard parmi les consoles possibles
- Comparez avec les vraies réponses dans `y_test` via la fonction `accuracy_score`

In [59]:
#@title Exercice 3.1

consoles = df_jeux.Platform.value_counts().index

for i in range(10):
  y_predictions_hasard = []

  for i in range(len(y_test)):
    y_predictions_hasard.append(np.random.choice(consoles))

  print(accuracy_score(y_test, y_predictions_hasard))

0.04475703324808184
0.03836317135549872
0.03580562659846547
0.043478260869565216
0.052429667519181586
0.049872122762148335
0.05370843989769821
0.0319693094629156
0.0370843989769821
0.05754475703324808


**RÉPONSE**

Si on répondait entièrement au hasard, on serait entre 3% et 6% de bonnes réponses.

33.5% est un peu décevant, mais c'est déjà mieux que le hasard.

**QUESTION**

Si on utilisait une approche différente : si on répondait toujours avec la console sur laquel le plus de jeux sont sortis, quelle aurait été l'exactitude?

In [60]:
#@title Exercice 3.2

plus_populaire = df_entrainement.Platform.value_counts().index[0]
print(plus_populaire)
y_predictions_plus_populaire = [plus_populaire] * len(y_test)

print(accuracy_score(y_test, y_predictions_plus_populaire))

PlayStation3
0.17774936061381075


**RÉPONSE**

En répondant toujours *PS3*, on a 17.8% de réponses correctes.

Vous devriez voir que finalement, en comparant avec des méthodes simples, 33.5% de bonnes réponses ce n'est pas aussi mauvais que ça aurait pu l'être.

On sait au moins que notre `KNN` a appris *un petit quelque chose*.

## Améliorer les résultats

Est-ce qu'on peut faire mieux que 33.5%? Probablement...

Réfléchissons un peu au fonctionnement de l'algorithme.

La notion de **distance entre deux observations** est centrale dans l'algorithme KNN :

$$\text{distance}(\vec{x}, \vec{y}) = \sqrt{(x_\text{1} - y_\text{1})^2 + \\ (x_\text{2} - y_\text{2})^2 + \\ (x_\text{3} - y_\text{3})^2 + \\ ~~~~~~... \\ (x_\text{N} - y_\text{N})^2}$$

**Question** : quelle est la distance entre :

- Minecraft,WiiU,2016,180000.0,90000.0,140000.0,30000.0

- Tetris,GameBoy,1989,23200000.0,2260000.0,4220000.0,580000.0


In [61]:
#@title Exercice 4.1

minecraft = df_jeux.loc[1750]
tetris = df_jeux.loc[5]

display(minecraft)
display(tetris)

### Écrivez une fonction qui calcule la distance entre deux jeux, avec la formule ci-haut
def distance(jeu1, jeu2):
  ...


########

# Solution

def distance(jeu1, jeu2):
  dist = np.sqrt(
      (jeu1.Year - jeu2.Year)**2 +
      (jeu1.NA_Sales - jeu2.NA_Sales)**2 +
      (jeu1.EU_Sales - jeu2.EU_Sales)**2 +
      (jeu1.JP_Sales - jeu2.JP_Sales)**2 +
      (jeu1.Other_Sales - jeu2.Other_Sales)**2
  )
  return dist

print(distance(minecraft, tetris))

,1750
Name,Minecraft
Platform,WiiU
Year,2016
NA_Sales,180000.0
EU_Sales,90000.0
JP_Sales,140000.0
Other_Sales,30000.0


,5
Name,Tetris
Platform,GameBoy
Year,1989
NA_Sales,23200000.0
EU_Sales,2260000.0
JP_Sales,4220000.0
Other_Sales,580000.0


23485702.033380415


Imaginons un nouveau jeu : Tetris 2, sorti sur Wii U

- Tetris   ,GameBoy,1989,23200000.0,2260000.0,4220000.0,580000.0
- Tetris 2,WiiU         ,**2015**,23200000.0,2260000.0,4220000.0,580000.0

La différence entre le vrai `Tetris` et notre `Tetris 2` imaginé est uniquement **l'année de sortie, en 2015**. On va imaginer que ce jeu est sorti sur *WiiU* (car il est sorti en 2015), mais qu'il a fait exactement les mêmes ventes que Tetris.

Affichez la distance entre `Tetris 2` et `Tetris`

In [62]:
#@title Exercice 4.2

tetris = df_jeux.loc[5]
tetris2 = pd.Series(
    ["Tetris 2", "WiiU", 2015, 23200000.0, 2260000.0, 4220000.0, 580000.0],
    index=tetris.index
)
####

display(tetris)
display(tetris2)
print(distance(tetris, tetris2))


,5
Name,Tetris
Platform,GameBoy
Year,1989
NA_Sales,23200000.0
EU_Sales,2260000.0
JP_Sales,4220000.0
Other_Sales,580000.0


,0
Name,Tetris 2
Platform,WiiU
Year,2015
NA_Sales,23200000.0
EU_Sales,2260000.0
JP_Sales,4220000.0
Other_Sales,580000.0


26.0


Imaginons encore un autre jeu : Minecraft 2, également sorti sur WiiU

- Minecraft  , WiiU,2016,180000.0,90000.0,140000.0,30000.0
- Minecraft 2,WiiU,2016,180000.0,90000.0,140000.0,300**30**.0

La seule différence entre le vrai `Minecraft` et notre `Minecraft 2` imaginé a été vendu à **seulement 30 exemplaires de plus** à travers le monde

Affichez la distance entre `Minecraft 2` et `Minecraft`

In [63]:
#@title Exercice 4.3

minecraft = df_jeux.loc[1750]
minecraft2 = pd.Series(
    ["Minecraft 2", "WiiU", 2016, 180000.0, 90000.0, 140000.0, 30030.0],
    index=tetris.index
)
####

display(minecraft)
display(minecraft2)
print(distance(minecraft, minecraft2))


,1750
Name,Minecraft
Platform,WiiU
Year,2016
NA_Sales,180000.0
EU_Sales,90000.0
JP_Sales,140000.0
Other_Sales,30000.0


,0
Name,Minecraft 2
Platform,WiiU
Year,2016
NA_Sales,180000.0
EU_Sales,90000.0
JP_Sales,140000.0
Other_Sales,30030.0


30.0


**QUESTION**

Regardez bien ces deux derniers résultats. Quelle est la distance la plus grande parmi:

- `Tetris` vs `Tetris 2` sorti 26 ans plus tard
- `Minecraft` vs `Minecraft 2` sorti la même année, mais vendu à 30 copies de plus

Est-ce que ce résultat fait du sens selon vous?

**RÉPONSE**

*Quelle est la distance la plus grande?*

=> Tetris est plus proche de Tetris 2 que Minecraft l'est de Minecraft 2

&nbsp;

*Est-ce que ça fait du sens?*

=> Explication dans la section suivante

## Mise à l'échelle des données

Il y a quelque chose qui cloche dans ces derniers résultats.

Faire passer l'année de 1989 à 2015, ça augmente *moins* la distance que quand on change le nombre d'exemplaires de +30 sur un total de 44 millions de copies vendues.

Pourtant, si je vous demandais de m'estimer la console d'un jeu seulement en vous basant sur le fait que l'année de sortie est en 2015, il y a très peu de chances que vous vous mélangiez entre le Game Boy original des années 80 et la Wii U...

&nbsp;

KNN est très sensible à des données qui sont **sur des échelles différentes**.

Ici on a :

- L'année de sortie, comprise entre 1983 et 2016.
- Le nombre de copies vendues, entre 10 000 et 41.5 millions

De base, la fonction de distance est autant affectée par un +1 année que par +1 copie vendue, ce qui n'a pas de sens.

&nbsp;

Beaucoup d'algorithmes d'apprentissage machine présument que les données seront **mises à l'échelle**.

Une façon de régler ce problème est de transformer chaque colonne pour les remettre dans un même intervalle, par exemple :

- L'année de sortie, modifiée pour être entre 0% (=1983) et 100% (=2016)
- Le nombre de copies vendues, entre 0% (=valeur min) et (100%=valeur max)

Changer l'année de +26 ans aurait un impact de +79% dans la colonne de l'année mise à l'échelle.

Changer le nombre de ventes Nord-Américaines de +30 aurait un impact de 30/4 millions => à peu près 0%.

In [64]:
display(X_entrainement.min())
display(X_entrainement.max())

,0
Year,1983.0
NA_Sales,10000.0
EU_Sales,10000.0
JP_Sales,10000.0
Other_Sales,10000.0


,0
Year,2016.0
NA_Sales,29080000.0
EU_Sales,11010000.0
JP_Sales,7200000.0
Other_Sales,10570000.0


## Code final avec les données ajustées

On peut utiliser la formule suivante sur chaque donnée, en utilisant le min et le max de sa colonne :

$$X = (X - min) / (max - min)$$

In [71]:
# On modifie les données pour que tout soit dans un même intervalle
# Pandas va faire ça une colonne à la fois
X_entrainement_echelle = (X_entrainement - X_entrainement.min()) / (X_entrainement.max() - X_entrainement.min())
X_test_echelle = (X_test - X_entrainement.min()) / (X_entrainement.max() - X_entrainement.min())

modele_jeux_avec_echelle = KNeighborsClassifier(5)

modele_jeux_avec_echelle.fit(X_entrainement_echelle, y_entrainement)

y_predictions = modele_jeux_avec_echelle.predict(X_test_echelle)

print("Exactitude:", accuracy_score(y_test, y_predictions))

Exactitude: 0.4616368286445013


On arrive à **~46%** d'exactitude

Beaucoup mieux!

## Analyser les erreurs

On a 46% d'exactitude **dans l'ensemble**.

On pourrait cependant avoir un modèle *biaisé* : la répartition des erreurs n'est peut-être pas égale.

&nbsp;

Considérez un modèle qui aurait la performance suivante :

- **Score parfait** sur tous les jeux de XBox360 (12.2% des données)
- **Score parfait** sur tous les jeux de PlayStation 1 2 et 3 (8.15%, 12.3% et 18.3% des données)
- **Toujours la mauvaise réponse pour toutes les autres consoles**.

Ce modèle aurait environ 51% d'exactitude...

&nbsp;

**Pour évaluer correctement un modèle, on doit regarder le type d'erreur qu'il commet.**

## Matrice de confusion

L'exactitude est une mesure de la performance incomplète.

Pour avoir une idée des erreurs commises, on peut utiliser la **matrice de confusion**.

&nbsp;

Il s'agit d'un tableau 2D `matrice[i, j]`, où :

- Le numéro de ligne $i$ est une classe prédite
- Le numéro de colonne $j$ est la classe réelle
- La valeur de `matrice[i, j]` est le nombre d'observations de classe $j$ que le modèle a prédit dans la classe $i$

Plus les valeurs sur la diagonale sont élevées (les cases où $i = j$), mieux c'est.

On peut regarder une ligne en particulier ou une colonne en particulier pour avoir une idée de la sorte d'erreur la plus commune.

&nbsp;

La matrice de confusion peut être calculée avec `sklearn` :

In [72]:
from sklearn.metrics import confusion_matrix

# Tableau numpy 2D:
matrice = confusion_matrix(y_test, y_predictions)

# On la transforme en DataFrame pour l'inspecter:
df_confusion = pd.DataFrame(
    matrice,
    columns=modele_jeux_avec_echelle.classes_,
    index=modele_jeux_avec_echelle.classes_
)
display(df_confusion)

,GameBoy,GameBoyAdvance,GameCube,Nintendo3DS,Nintendo64,NintendoDS,NintendoNES,PlayStation,PlayStation2,PlayStation3,...,PlayStationVita,SegaDreamCast,SegaGenesis,SegaSaturn,SuperNintendo,Wii,WiiU,XBox,XBox360,XBoxOne
GameBoy,3,1,1,0,1,1,1,4,3,0,...,0,0,0,0,6,0,0,0,0,0
GameBoyAdvance,0,6,8,0,0,1,0,1,4,0,...,0,0,0,0,0,0,0,0,0,0
GameCube,0,7,5,0,1,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
Nintendo3DS,0,0,0,20,0,2,0,0,0,15,...,4,0,0,0,0,1,1,0,1,0
Nintendo64,0,0,0,0,3,0,0,11,3,0,...,0,0,0,0,0,0,0,0,0,0
NintendoDS,0,2,0,0,0,25,0,0,4,10,...,0,0,0,0,0,5,0,0,5,0
NintendoNES,2,0,0,0,0,0,11,0,0,0,...,0,0,0,0,3,0,0,0,0,0
PlayStation,1,0,0,0,4,0,0,52,4,0,...,0,0,0,0,1,0,0,0,0,0
PlayStation2,0,9,7,0,1,11,0,3,57,2,...,0,0,0,0,0,2,0,2,3,0
PlayStation3,0,0,0,7,0,7,0,0,3,77,...,4,0,0,0,0,6,2,0,15,0


Plus souvent qu'autrement, on veut également regarder les **pourcentages dans chaque catégorie prédite**.

On peut visualiser ça avec une carte thermique.

In [73]:
from sklearn.metrics import confusion_matrix

# Tableau numpy 2D:
matrice = confusion_matrix(y_test, y_predictions, normalize='pred')

# On la transforme en DataFrame pour l'inspecter:
df_confusion = pd.DataFrame(
    matrice,
    columns=modele_jeux_avec_echelle.classes_,
    index=modele_jeux_avec_echelle.classes_
)

# Visualisation sous forme de carte thermique
display(px.imshow(df_confusion,
                  text_auto=True, title="Matrice de confusion (en pourcentages)",
                  labels={"x": "Vraie classe", "y": "Prediction"}))

**QUESTION**

Regardez **la diagonale** de la matrice de confusion et dites quelles sont les 3 consoles les mieux reconnues

**RÉPONSE**

Top 3 :

- NintendoNES a 68.75% de bons scores
- PlayStation a le meilleur score (70%)
- SegaGenesis a 100% de bons scores


**QUESTION**

Pour les jeux qui sont sortis sur SuperNintendo ont beaucoup de difficulté à se faire reconnaître...

Expliquez ce qui se passe en regardant quelles classes sont prédites lorsqu'on a un jeu de SuperNintendo.

**RÉPONSE**

Les jeux de SuperNintendo se font classer avec d'autres jeux de la même époque : les jeux de NES et les jeux de GameBoy.

Ces deux consoles ont un overlap dans le nombre de

**Question de réflexion**

Qu'est-ce qui se passe avec les trois consoles Sega?

- SegaDreamCast et SegaSaturn ont 0% de bonnes réponses
- SegaGenesis a 100% de bonnes réponses

Comment expliquez-vous ces scores? Retournez voir le nombre de jeux sur chacune de ces trois plateformes pour répondre à la question

**RÉPONSE**

Les chiffres sont extrêmes, mais ça semble être des faux estimés des scores : ces trois consoles ont très très peu d'exemples...

En mélangeant le `DataFrame` pour l'entraînement, il semblerait qu'on a **aucun exemple de DreamCast ou de Saturn dans les `y_test`**, ce qui empêche de tester correctement ces résultats.

Pour SegaGenesis, on a probablement simplement été chanceux. Des nouveaux jeux de SegaGenesis qui ne faisaient pas partie du jeu de données utilisé ne seront clairement pas bien reconnus.

## Quoi faire ensuite?

Quand notre modèle n'est pas assez performant à notre goût, on a plusieurs options :

- Collecter plus de données et voir si ça aide
- Modifier le K utilisé pour autre chose que 5
- Changer d'algorithme (KNN est un seul algo parmi plusieurs possibles!)
- Faire de l'**ingénérie de caractéristiques** : ajouter de l'information supplémentaite dans notre jeu de données selon des connaissances spécifiques au domaine

Une idée : on veut deviner la console à partir des caractéristiques, mais on a supprimé une colonne assez importante... Le nom du jeu!

Si je vous parle des ces exemples :

- Wii Sports Resort,à2009,15750000.0,11010000.0,3280000.0,2960000.0
- Wii Play,Wii,2006,14030000.0,9200000.0,2930000.0,2850000.0
- New Super Mario Bros. Wii,2009,14590000.0,7060000.0,4700000.0,2260000.0
- Wii Fit,2007,8940000.0,8029999.999999999,3600000.0,2150000.0

Seriez-vous capables de deviner la console?

Essayez la chose suivante : ajoutez une colonne `True/False` qui indique `True` si "Wii" est contenu dans le nom du jeu, puis réentraînez votre modèle. Utilisez la matrice de confusion pour voir si ça a aidé ou non.

Essayez d'imaginer d'autres mots-clés qui pourraient aider à la classification.

In [74]:
#@title Exercice bonus 1

### Testez d'autres valeurs de K pour essayer d'améliorer la performance

In [75]:
#@title Exercice bonus 2

###

# Idées : les noms des consoles (ex.: Wii, 64, "Super" pour les jeux sur Super Nintendo...)
# les noms de franchises (ex.: Halo, Mario, Sonic, Pokemon, ...)

## Considérations finales

Rendu ici, une fois qu'on a une bonne idée de la performance de notre algorithme après les améliorations (46%), on pourrait recommencer un dernier entraînement sur toutes les données et utiliser cette version finale dans un vrai projet.

On saurait alors que notre modèle a une exactitude d'au-moins ≈46%, peut-être plus si on a de la chance et que le dernier tiers de données ajouté aide à reconnaître d'autres jeux.

# En résumé, la classification

- **Tâche de Classification** : trouver à quelle catégorie appartient une certaine observation
- Notion de distance entre deux observations
- Algorithme KNN : trouver les K plus proches voisins de notre observation et les faire voter sur la classe
- Évaluer le résultat : il faut séparer le jeu de données en deux, `df_entrainement` et `df_test`
- Évaluer le résultat : l'**exactitude** d'un modèle de classification est son pourcentage de bonnes réponses
- Utiliser `sklearn` pour faire ça sur n'importe quel jeu de données
- Modifier les données avec une mise à l'échelle, c'est généralement essentiel pour avoir des bons résultats
- La matrice de confusion peut nous aider à comprendre le genre d'erreur que notre modèle fait